<a href="https://colab.research.google.com/github/perfectism13/learning_colab/blob/master/flow_xception_model/flow_xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###挂载google drive和导入库

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir(r'/content/drive/My Drive/data/imgs/')
print(os.getcwd())
!ls
!pip install keras==2.1.5
import keras
print(keras.__version__)
!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/data/imgs
'(1-726)clean.mp4'	        my_model12.h5
'(1-726).mp4'		        my_model13.h5
'（726-800)clean.mp4'	        my_model1.h5
 （726-800）.mp4	        my_model9.h5
'(726-966)clean.mp4'	        sample_submission.csv
'(726-966).mp4'		        test
 driver_imgs_list_class_2.csv   train
 driver_imgs_list_class.csv     valid
 driver_imgs_list.csv	        vect
 driver_imgs_list_right.csv     video_model_change_vd_data_aug.h5
 flow_xception.h5	        video_model_change_vd_data_aug_modified.h5
 my_model10.h5		        video_model_change_vd.h5
 my_model11.h5		        video_model.h5


Using TensorFlow backend.


2.1.5
Sat Feb  8 08:40:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
| 

In [0]:
import os
os.environ['KERAS_BACKEND']='tensorflow'
import keras

In [0]:
import numpy as np
np.random.seed(2017)

import os
import glob
import cv2
import math
import pickle
import pandas as pd
import random
from tqdm import tqdm
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, Adam, Nadam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss

### 切分数据
按照司机的id来切分训练数据集。把driver_imgs_list分成两个列表。一个是训练列表，里面是所有训练集司机的图片文件名。另一个是验证列表，里面是所有验证集司机的图片文件名。然后把两个列表都随机打乱。 这里列表里面保存的都只是excel文件里的一行行数据（包括文件名，分类，司机id），而不是图片本身。等训练时通过生成器读取图片，这样节约内存。 另外还保存了一个类别字典，便于以后从同一类中抽取图片进行拼接。

In [4]:
# divide drivers
# unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075']
# unique_list_valid = ['p016', 'p021', 'p022', 'p024']
unique_list_train = ['p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075']
unique_list_valid = ['p002', 'p012', 'p014', 'p015']
#print (unique_list_train, unique_list_valid)

# get index: driver_id, class, image name
index = os.path.join('.', 'driver_imgs_list_class_2.csv')

# build the driver id dictionary and class dictionary
f = open(index, 'r')
id_dict = dict()
class_dict = dict()
lines = f.readlines()
# for line in lines[1:]:
#     arr = line.strip().split(',')
#     if arr[0] not in id_dict.keys():
#         id_dict[arr[0]] = [line]
#     else:
#         id_dict[arr[0]].append(line)
#     if arr[1] not in class_dict.keys():
#         class_dict[arr[1]] = [line]
#     else:
#         class_dict[arr[1]].append(line)
# f.close()

# split the train list and valid list by id
train_list = []
valid_list = []
# for id in id_dict.keys():
#     if id in unique_list_train:
#         train_list.extend(id_dict[id])
#     elif id in unique_list_valid:
#         valid_list.extend(id_dict[id])

# for cla in class_dict.keys():
#   for id in id_dict.keys():
#     if id in unique_list_train:
#         train_list.extend(class_dict[cla])
#     elif id in unique_list_valid:
#         valid_list.extend(class_dict[cla])
for line in lines[1:]:
    arr = line.strip().split(',')
    if arr[0]  in unique_list_train:
        train_list.append(line)
    elif arr[0]  in unique_list_valid:
      valid_list.append(line)
f.close()

random.shuffle(train_list)
random.shuffle(valid_list)

print (len(train_list), len(valid_list))

18302 3299


### 获取test set图片的列表
获取sample_submission.csv中所有测试图片的文件名，预测test set时使用。

In [5]:
test_index = os.path.join('.', 'sample_submission.csv')
f = open(test_index, 'r')
lines = f.readlines()
test_list = []
for line in lines[1:]:
    arr = line.strip().split(',')
    test_list.append(arr[0])
f.close()
print (test_list[:5])

['img_1.jpg', 'img_10.jpg', 'img_100.jpg', 'img_1000.jpg', 'img_100000.jpg']


### 转换为One Hot Encode标签

对分类标签进行One Hot Encode的函数如下。

In [6]:
# one hot encode the class label

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])   
def one_hot_encode(x):
    return lb.transform(x)
t = one_hot_encode(['c1', 'c2'])
print(t)

[[0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]]


###获取光流

In [0]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
from keras.preprocessing import image

# image rotation
def rotate(x1,x2, degree, row_axis=0, col_axis=1, channel_axis=2, fill_mode='wrap', cval=0.):
    theta = np.pi / 180 * degree
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]])
    h, w = x1.shape[row_axis], x1.shape[col_axis]
    transform_matrix = image.transform_matrix_offset_center(rotation_matrix, h, w)
    x1 = image.apply_transform(x1, transform_matrix, channel_axis, fill_mode, cval)
    x2 = image.apply_transform(x2, transform_matrix, channel_axis, fill_mode, cval)
    return x1,x2

# image shift
def shift(x1,x2, wshift, hshift, row_axis=0, col_axis=1, channel_axis=2, fill_mode='wrap', cval=0.):
    h, w = x1.shape[row_axis], x1.shape[col_axis] #读取图片的高和宽
    tx = hshift * h #高偏移大小，若不偏移可设为0，若向上偏移设为正数
    ty = wshift * w #宽偏移大小，若不偏移可设为0，若向左偏移设为正数
    translation_matrix = np.array([[1, 0, tx],
                                  [0, 1, ty],
                                  [0, 0, 1]])
    transform_matrix = translation_matrix  
    x1 = image.apply_transform(x1, transform_matrix, channel_axis, fill_mode, cval)
    x2 = image.apply_transform(x1, transform_matrix, channel_axis, fill_mode, cval)
    return x1,x2

def get_flow_aug(path1,path2):
  frame1 = cv2.imread(path1)
  frame1 = cv2.resize(frame1, (299, 299))
  prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
  hsv = np.zeros_like(frame1)
  hsv[...,2] = 255
  frame2 = cv2.imread(path2)
  frame2 = cv2.resize(frame2, (299, 299))
  #random rotate
  deg = random.uniform(-10, 10) #random rotate limit
  frame1,frame2 = rotate(frame1,frame2, deg)
  #random shift
  wshift = random.uniform(-0.1, 0.1)
  hshift = random.uniform(-0.1, 0.1)
  frame1,frame2 = shift(frame1,frame2, wshift, hshift)
  next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
  flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
  mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
  hsv[...,0] = ang*180/np.pi/2
  hsv[...,1] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
  flow = cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
  # normalization
  flow = np.array(hsv, dtype=np.float32)  
  flow /= 127.5
  flow -= 1.
  return flow

def get_flow(path1,path2):
  frame1 = cv2.imread(path1)
  frame1 = cv2.resize(frame1, (299, 299))
  prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
  hsv = np.zeros_like(frame1)
  hsv[...,2] = 255
  frame2 = cv2.imread(path2)
  frame2 = cv2.resize(frame2, (299, 299))
  next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
  flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
  mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
  hsv[...,0] = ang*180/np.pi/2
  hsv[...,1] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
  flow = cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
  # normalization
  flow = np.array(flow, dtype=np.float32)  
  flow /= 127.5
  flow -= 1.
  return flow


### 训练图片生成器

从训练列表中遍历。yield一个batch的训练图片及其标签。图片经过了实时增强。另外还有50%的概率随机选取另一张同类里的图片，将两张的左右各半边拼接在一起。这也是为了训练模型对分类的关键部位进行学习，而不是记住司机的样子

In [0]:
# define my train data generator

def train_gen(batch_size):
    #random.shuffle(train_list) # 每一代都随机shuffle训练集
    current = 0
    while 1:
        x = []
        y = []
        while len(y) < batch_size:
            line = train_list[current]
            arr = line.strip().split(',')
            path1 = os.path.join('.', 'train', str(arr[1]), str(arr[2]))
            path3 = os.path.join('.', 'train', str(arr[4]), str(arr[5]))
            # img = get_im_cv2_aug(path1, 299)
            # img3 = get_im_cv2_aug(path3, 299)
            mffs = get_flow_aug(path1,path3)
            # if random.random()>0.5:
            #     line2 = random.choice(class_dict[arr[1]])
            #     bname = line2.strip().split(',')[2]
            #     path2 = os.path.join('.', 'train', str(arr[1]), str(bname))
            #     img2 = get_im_cv2_aug(path2, 299)
            #     left = img[:, :150, :]
            #     right = img2[:, 150:, :]
            #     img = np.concatenate((left, right), axis=1)
            x.append(mffs)
            label = one_hot_encode([str(arr[1])])[0]
            y.append(label)
            current += 1
            if current+1 >= len(train_list):
                current = 0
        x = np.array(x)
        img_size = 299
        x = x.reshape(batch_size, img_size, img_size, 3) 
        y = np.array(y, dtype = np.uint8)
        y = y.reshape(batch_size, 10)

        yield (x, y)

### 验证图片生成器

从验证列表中遍历。yield一个batch的验证图片及其标签。为了体现模型训练后的拟合能力，图片没有进行实时增强。

In [0]:
# define my validation data generator

def valid_gen(batch_size):
    current = 0
    while 1:
        x = []
        y = []
        while len(y) < batch_size:
            line = train_list[current]
            arr = line.strip().split(',')
            path1 = os.path.join('.', 'train', str(arr[1]), str(arr[2]))
            path3 = os.path.join('.', 'train', str(arr[4]), str(arr[5]))
            #print (path)
            mffs = get_flow(path1,path3)
            x.append(mffs)
            label = one_hot_encode([str(arr[1])])[0]
            y.append(label)
            current += 1
            if current+1 >= len(valid_list):
                current = 0
        x = np.array(x)
        img_size = 299
        x = x.reshape(batch_size, img_size, img_size, 3) 
        y = np.array(y, dtype = np.uint8)
        y = y.reshape(batch_size, 10)
        yield (x, y)

### 构建模型

用keras构建模型。使用在ImageNet上预训练好的Xception模型，接上一个global average pooling层，dropout防止过拟合，最后一个全连接层输出10个类别的概率。在全连接层的权重上采用了L2正则化。锁定模型的前70层不更新权重.

In [10]:
from keras import backend as K
K.clear_session()

In [11]:
# new model

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

img_size = 299
x = Input((img_size, img_size, 3))
base_model = Xception(input_tensor = x, weights='imagenet', include_top=False)


x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(x)
model = Model(base_model.input, x)

model.load_weights('flow_xception.h5')
for i in range(70):
    model.layers[i].trainable = False
    
model.summary()
len(model.layers)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0]

135

### 训练
进行模型训练。这里batch size为64。用了自适应优化器Nadam，使用schedule learning rate自动调整学习率的方法并在验证loss不下降时early stopping。一共训练5代，最后的验证集loss仍然有下降空间。steps per epoch设定为在一个epoch内所有训练图片被遍历1次.

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
model.compile(optimizer=Nadam(),loss='categorical_crossentropy',metrics=['categorical_accuracy'])

def learning_rate(epoch): 
    ini_lr = 0.002
    lr = ini_lr * pow(10, -epoch)
    return lr

cp = ModelCheckpoint(filepath="flow_xception.h5", save_best_only=True)
es = EarlyStopping()
lrs = LearningRateScheduler(learning_rate)
hist = model.fit_generator(train_gen(64), steps_per_epoch = 286, epochs = 5, workers=4, max_q_size=20, use_multiprocessing=True, validation_data = valid_gen(64), validation_steps = 52, callbacks=[cp, es, lrs])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=286, epochs=5, workers=4, use_multiprocessing=True, validation_data=<generator..., validation_steps=52, callbacks=[<keras.ca..., max_queue_size=20)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
285/286 [============================>.] - ETA: 6s - loss: 0.6646 - categorical_accuracy: 0.8048 

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2330: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


286/286 [==============================] - 2152s 8s/step - loss: 0.6647 - categorical_accuracy: 0.8047 - val_loss: 2.4030 - val_categorical_accuracy: 0.3702
Epoch 2/5
 98/286 [=========>....................] - ETA: 28:47 - loss: 0.6044 - categorical_accuracy: 0.8284

### 模型结果

保存模型。输出history数据，观察loss曲线。 这里忘记设置inline，没有显示曲线。从数据来看验证集loss明显降低，分类精度提高。

### 对测试集进行预测

载入模型。对test数据进行预测，将结果写入csv文件。这里使用了keras自带的图片生成器。

In [0]:
# load the model 载入已经训练好的模型！！！
from keras.models import load_model  
model = load_model('my_model12.h5')

In [0]:
# define my test data generator
from keras.preprocessing.image import *
from keras.applications.xception import preprocess_input

test_index = os.path.join('.', 'test')
gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = gen.flow_from_directory(test_index, target_size=(299,299), shuffle=False, batch_size=64, class_mode=None)


In [0]:

y_pred = model.predict_generator(test_gen, steps=test_gen.samples//64+1)

In [0]:
test_id = []
for f in test_gen.filenames:
    test_id.append(f)
    
result = pd.DataFrame(y_pred, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
result.insert(0, 'img', pd.Series(test_id, index=result.index))
result.to_csv(os.path.join('.' ,'my_submission12.csv'), index=False)

### 模型成绩
提交kaggle成绩：public score 0.50 private score 0.36